# Forward mapping of articulation and acoustics

## Refs:
- Slider: https://plot.ly/~jackluo/2228/moving-sma-slider-visualization-with-plo/#/
- Curve fitting: https://plot.ly/~rreusser/90/chebyshev-points-for-least-squares-curve/#/
- Subplots: https://plot.ly/python/subplots/
- Mixed subplots: https://plot.ly/python/mixed-subplots/

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
init_notebook_mode(connected=True)

import time

In [2]:
# Load single speaker data
df = pd.read_pickle('JW12.pckl')

In [3]:
df.head()

,Subj,FName,PreLab,NexLab,Label,Context,Dur,T1x,T1y,T2x,...,segEnd,GONS,NONS,MAXC,NOFFS,GOFFS,GesTarDur,timeMidWithinGest,timeMid2GestDiff,GestFound
0,JW12,JW12_TP007,P,N,AE1,P-AE1-N,0.119728,-15.086,-1.4980,-29.573500,...,5.962424,5.801770,5.884162,5.904760,5.918492,5.966554,0.034330,1.0,0.000000,1
1,JW12,JW12_TP010,HH,D,AE1,HH-AE1-D,0.099773,-17.583,3.0545,-30.327500,...,6.339805,6.076410,6.186266,6.206864,6.241194,6.378514,0.054928,0.0,0.048724,1
2,JW12,JW12_TP017,N,S,AE1,N-AE1-S,0.119728,-10.927,-0.0570,-25.707001,...,1.635277,1.380066,1.510520,1.592912,1.613510,1.682170,0.102990,1.0,0.000000,1
3,JW12,JW12_TP017,K,R,AE1,K-AE1-R,0.079819,-22.037,12.3780,-33.766001,...,2.283803,2.100996,2.300110,2.320708,2.348172,2.444296,0.048062,0.0,0.056216,1
4,JW12,JW12_TP017,IY0,N,AE1,IY0-AE1-N,0.049887,-15.916,7.5980,-29.587999,...,2.583123,2.492358,2.602214,2.615946,2.636544,2.705204,0.034330,0.0,0.044035,1


In [4]:
# Load pal, pha
with open('pal_pha.pckl', 'rb') as pckl:
    pal, pha = pickle.load(pckl)
# Set parameters
artic_col = ['T1x', 'T1y', 'T2x', 'T2y', 'T3x', 'T3y',
             'T4x', 'T4y', 'ULx', 'ULy', 'LLx', 'LLy', 'MNIx', 'MNIy']
acous_col = ['F1', 'F2']
vowel_list = ['AE1', 'AH1', 'AO1', 'EH1', 'IH1', 'AA1', 'IY1', 'UW1', 'UH1']

# Prepare dataset
X_raw = df.loc[:, artic_col].copy().as_matrix()
Y_raw = df.loc[:, acous_col].copy().as_matrix()
# Standardize before PCA
# Articulation
X_scaler = StandardScaler().fit(X_raw)
X_std = X_scaler.transform(X_raw)  # cf .inverse_transform()
# Acoustics
Y_scaler = StandardScaler().fit(Y_raw)
Y_std = Y_scaler.transform(Y_raw)

# PCA
pca = PCA(n_components=3)
pca.fit(X_std)
X_reduced = pca.transform(X_std)

# Linear Regression
#  X*w = y
W = np.dot(np.linalg.pinv(X_reduced), Y_std)

# Compute median articulation & acoustic values
medianArtic = np.zeros((len(vowel_list), 14))
medianAcous = np.zeros((len(vowel_list), 2))
for i, v in enumerate(vowel_list):
    x = df.loc[df.Label == v, artic_col].as_matrix()
    y = df.loc[df.Label == v, acous_col].as_matrix()
    medianArtic[i, :] = np.median(x, axis=0)  # 7x14
    medianAcous[i, :] = np.median(y, axis=0)  # 7x2

# Estimate F1, F2 for each vowel
y_scaled_vowels = np.dot(pca.transform(X_scaler.transform(medianArtic)), W)
y_vowels = Y_scaler.inverse_transform(y_scaled_vowels)  # 7x2

In [51]:
# PCA space
trace1 = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=5,
        symbol='circle',
        color='rgb(0,0,217)',
        opacity=0.8
    ),
)

# Articulator space
# pharynx
trace2 = go.Scatter(
    x=pha[:,0],
    y=pha[:,1],
    mode='lines',
    name='Pharynx',
    xaxis='x2',
    yaxis='y2',
)
# palate
trace3 = go.Scatter(
    x=pal[:,0],
    y=pal[:,1],
    mode='lines',
    name='Palate',
    xaxis='x2',
    yaxis='y2',
)
# Reference formants
trace4 = go.Scatter(
    x=[0],
    y=[0],
    name='Formant',
    xaxis='x3',
    yaxis='y3',
)
# Dynamic formants


data = [trace1,trace2,trace3,trace4]

In [166]:
layout = go.Layout(
    title='Articulation to Acoustics',
    height=300,
    margin=dict(
    l=30,
    r=30,
    b=70,
    t=70),
    showlegend=False,
    
    scene = dict(
        xaxis=dict(
            title='',
            range=[-4,4],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        yaxis=dict(
            title='',
            range=[-4,4],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        zaxis=dict(
            title='',
            range=[-4,4],
            dtick=1,
            tickfont=dict(
                size=10,
            ),
        ),
        domain=dict(
            x=[0,0.26],
        ),
        camera=dict(
            eye=dict(x=1.5,y=1.5,z=0.8)
        ),
    ),
    xaxis2=dict(
        title='Midsagittal view',
        domain=[0.36,0.62],
        range=[-90,40],
        dtick=20,
        tickangle=0,
        tickfont=dict(
            size=10,
        ),
    ),
    yaxis2=dict(
        range=[-30,30],
        dtick=10,
    ),
    xaxis3=dict(
        title='Formant space',
        domain=[0.72,0.98]
    )
)

In [167]:
annotations = dict(
    text='PCA space',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0.1,
    y=-0.4,
)

layout['annotations'] = [annotations]

In [168]:
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='forward_mapping')